In [1]:
## conic dataset
# read in npy data and store them as images (png) and labels (tiff)
#%%
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
from skimage.measure import label

# 1: Neoplastic 
# 2: Inflammatory
# 3: Connective
# 4: Dead 
# 5: Epithelial

# folder with original dataset
rootfolder = "E:\Pannuke"
# folder where to store labels and images as tiff and png
outputfolder = "D:/NucleiSegmentation/Projects/NCIS/NeuronInstanceSeg-master/NeuronInstanceSeg-master/datasets/Pannuke/40x/256x256"

foldlist_ = ["fold_1", "fold_2","fold_3"]
foldlist = ["fold1", "fold2","fold3"]
# E:\Pannuke\fold_1\images\fold1

typesall = []
namesall = []
foldsall = []

for j, foldj in enumerate(foldlist):
    foldi = foldlist_[j]
    images = np.load(os.path.join(rootfolder,foldi,"images",foldj,'images.npy'))
    types = np.load(os.path.join(rootfolder,foldi,"images",foldj,'types.npy'))
    masks = np.load(os.path.join(rootfolder,foldi,"masks",foldj,'masks.npy'))
    typesall.extend(list(types))
    for i in np.arange(images.shape[0]):
        cv2.imwrite(os.path.join(outputfolder,'image', "pannuke_"+types[i].lower() +"_"+foldj+"_"+str(i)+'.png'), cv2.cvtColor(images[i].astype(np.uint8), cv2.COLOR_RGB2BGR))
        classes = np.zeros([256,256])
        # this does nto work because there are pixels in pannuke that are assigned to more than one class
        # so the sum leads to non existing classes (e.g., class 8)
        #classes = np.sum(np.ones([256,256,5])*np.array([1,2,3,4,5])*((masks[i,:,:,0:5]>0)).astype(int), axis = -1)
        for c in np.arange(5):
            classes[masks[i,:,:,c]>0]=c+1
        cv2.imwrite(os.path.join(outputfolder,'class', "pannuke_"+types[i].lower() +"_"+foldj+"_"+str(i)+'.tiff'), classes)
        labels = label(np.sum(masks[i,:,:,0:5],axis = -1).astype(int),connectivity=1)
        cv2.imwrite(os.path.join(outputfolder,'label',  "pannuke_"+types[i].lower() +"_"+foldj+"_"+str(i)+'.tiff'), labels)
        namesall.append("pannuke_"+types[i].lower() +"_"+foldj+"_"+str(i))
        foldsall.append(foldj)

# for i, filename in enumerate(file_names):
#     cv2.imwrite(os.path.join(outputfolder,'label', filename+"_"+foldi+'.tiff'), labels[i,:,:,0])
#     # open cv assumes images on disk to be rgb and manage them in memory as bgr
#     # when writing on disk, it assumes the image in memory (to be written onto disk) to be bgr, so we need to voncert it first
#     cv2.imwrite(os.path.join(outputfolder,'image', filename+"_"+foldi+'.png'), cv2.cvtColor(images[i], cv2.COLOR_RGB2BGR))
#     # save class labels
#     cv2.imwrite(os.path.join(outputfolder,'class', filename+'.tiff'), labels[i,:,:,1])
# #cv2.imwrite(os.path.join(label_folder, image.split('.p')[0].split('\\')[-1]+'.tiff'), img)
# #%%

In [2]:
# dictionary of lists 
dict = {'name': namesall, 'tissue': typesall, 'fold_original': foldsall} 
    
df = pd.DataFrame(dict)
    
print(df)
df.to_csv('pannuke_info.csv', index=False)

                          name  tissue fold_original
0       pannuke_breast_fold1_0  Breast         fold1
1       pannuke_breast_fold1_1  Breast         fold1
2       pannuke_breast_fold1_2  Breast         fold1
3       pannuke_breast_fold1_3  Breast         fold1
4       pannuke_breast_fold1_4  Breast         fold1
...                        ...     ...           ...
7896  pannuke_colon_fold3_2717   Colon         fold3
7897  pannuke_colon_fold3_2718   Colon         fold3
7898  pannuke_colon_fold3_2719   Colon         fold3
7899  pannuke_colon_fold3_2720   Colon         fold3
7900  pannuke_colon_fold3_2721   Colon         fold3

[7901 rows x 3 columns]
